In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv("Plant_1_Generation_Data.csv")
df2 = pd.read_csv("Plant_1_Weather_Sensor_Data.csv")

In [3]:
df1['DATE_TIME'] = df1['DATE_TIME'].astype('datetime64[ns]')
df2['DATE_TIME'] = df2['DATE_TIME'].astype('datetime64[ns]').replace('/','-')

df = pd.merge(df1,df2, how='left', on='DATE_TIME')

In [4]:
df['DATE'] = df['DATE_TIME'].dt.date

df = df.drop(columns=['PLANT_ID_x', 'SOURCE_KEY_x','PLANT_ID_y', 'SOURCE_KEY_y'])

df.head()

,DATE_TIME,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,AMBIENT_TEMPERATURE,MODULE_TEMPERATURE,IRRADIATION,DATE
0,2020-05-15,0.0,0.0,0.0,6259559.0,25.184316,22.857507,0.0,2020-05-15
1,2020-05-15,0.0,0.0,0.0,6183645.0,25.184316,22.857507,0.0,2020-05-15
2,2020-05-15,0.0,0.0,0.0,6987759.0,25.184316,22.857507,0.0,2020-05-15
3,2020-05-15,0.0,0.0,0.0,7602960.0,25.184316,22.857507,0.0,2020-05-15
4,2020-05-15,0.0,0.0,0.0,7158964.0,25.184316,22.857507,0.0,2020-05-15


In [5]:
def filling_droping(x):
    x = x[x['IRRADIATION'] != 0.0]
    
    x = x.fillna(method = 'ffill')
    
    return x

In [6]:
from sklearn import set_config
set_config('diagram')